In [1]:
import math
import csv
import pandas as pd
import matplotlib.pyplot as plt

# Nama file CSV untuk menyimpan basis kasus
case_base_file = 'crop_recommendation.csv'

# Fungsi untuk membaca basis kasus dari file CSV
def load_case_base(file_path):
    case_base = []
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            case_base.append({
                "criteria": {
                    "N": float(row["N"]),
                    "P": float(row["P"]),
                    "K": float(row["K"]),
                    "temperature": float(row["temperature"]),
                    "humidity": float(row["humidity"]),
                    "ph": float(row["ph"]),
                    "rainfall": float(row["rainfall"])
                },
                "label": row["label"]
            })
    return case_base

# Fungsi untuk mengonversi kriteria menjadi vektor numerik
def criteria_to_vector(criteria):
    return [
        criteria["N"],
        criteria["P"],
        criteria["K"],
        criteria["temperature"],
        criteria["humidity"],
        criteria["ph"],
        criteria["rainfall"]
    ]

# Fungsi untuk menghitung Cosine Similarity antara dua vektor
def cosine_similarity(vector1, vector2):  
    dot_product = sum(v1 * v2 for v1, v2 in zip(vector1, vector2))
    magnitude1 = math.sqrt(sum(v1 ** 2 for v1 in vector1))
    magnitude2 = math.sqrt(sum(v2 ** 2 for v2 in vector2))
    
    if magnitude1 == 0 or magnitude2 == 0: 
        return 0
    
    return dot_product / (magnitude1 * magnitude2)

# Fungsi untuk mencari kasus yang paling mirip menggunakan Cosine Similarity
def find_most_similar_case(case_base, new_criteria):
    most_similar_case = None
    max_similarity = 0
    similarity_results = []
    
    new_criteria_vector = criteria_to_vector(new_criteria)

    for i, case in enumerate(case_base, start=1):
        case_vector = criteria_to_vector(case["criteria"])
        similarity = cosine_similarity(new_criteria_vector, case_vector)
        
        # Simpan hasil similarity dan rekomendasi ke dalam list
        similarity_results.append({
            "Case ID": i,
            "Criteria": case["criteria"],
            "Label": case["label"],
            "Similarity": similarity
        })
        
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_case = case
            
    return most_similar_case, similarity_results, max_similarity

In [2]:
# Fungsi untuk memberikan rekomendasi berdasarkan kasus yang paling mirip
def label_crop(case_base, new_criteria):
    similar_case, similarity_results, max_similarity = find_most_similar_case(case_base, new_criteria)
    
    # Buat DataFrame dan urutkan berdasarkan nilai similarity tertinggi
    df = pd.DataFrame(similarity_results)
    df = df.sort_values(by="Similarity", ascending=False).reset_index(drop=True)
    df.index += 1  # ranking berdasarkan index
   
    print("\nTabel Hasil Similaritas dan Ranking:")
    print(df[["Case ID", "Criteria", "Label", "Similarity"]])  # Tampilkan tabel dengan kolom yang diinginkan

    label = None    

    if max_similarity >= 0.95:
        label = similar_case["label"]
        print(f"\nRekomendasi awal berdasarkan kasus paling mirip: {label}")
    else:
        print("\nNilai similarity tidak memenuhi ambang batas (threshold).")
        # Revisi
        new_label = input("Masukkan rekomendasi baru untuk kasus ini: ").strip()
        print(f"Rekomendasi telah direvisi menjadi: {new_label}")
        # Revisi & Retain
        label = retain(case_base, new_criteria, new_label)
    return label

In [3]:
# Fungsi untuk menyimpan kasus baru ke basis kasus
def retain(case_base, new_criteria, new_label):
    new_case = {
            "N": new_criteria["N"],
            "P": new_criteria["P"],
            "K": new_criteria["K"],
            "temperature": new_criteria["temperature"],
            "humidity": new_criteria["humidity"],
            "ph": new_criteria["ph"],
            "rainfall": new_criteria["rainfall"],
            "label": new_label
            }

    # Tambahkan kasus baru ke basis kasus
    case_base.append(new_case)

    try:
        save_case_to_csv(case_base_file, new_case)
        print("Kasus baru telah disimpan ke basis kasus.")
    except Exception as e:
        print(f"Terjadi kesalahan saat menyimpan ke CSV: {e}")

    return new_label

In [4]:
# Fungsi untuk menambah kasus baru ke file CSV
def save_case_to_csv(file_path, new_case):
    try:
        with open(file_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([
                new_case["N"],
                new_case["P"],
                new_case["K"],
                new_case["temperature"],
                new_case["humidity"],
                new_case["ph"],
                new_case["rainfall"],
                new_case["label"]
            ])
        print(f"Kasus baru telah disimpan ke dalam {file_path}.")
    except KeyError as e:
        print(f"Kesalahan: key yang diminta tidak ditemukan di new_case. Key error: {e}")
    except Exception as e:
        print(f"Terjadi kesalahan lain: {e}")

In [5]:
# Penggunaan
case_base = load_case_base(case_base_file)

# Kriteria baru untuk rekomendasi tanaman
new_case_crop = {
    "N": 115,
    "P": 221,
    "K": 3,
    "temperature": 4.5522,
    "humidity": 66.71273555,
    "ph": 8.8,
    "rainfall": 99.101211
}

# Rekomendasi awal
label = label_crop(case_base, new_case_crop)  # retain sudah dipanggil di dalam fungsi ini jika diperlukan
print(f"Label yang cocok untuk kasus baru {new_case_crop}: {label}")



Tabel Hasil Similaritas dan Ranking:
      Case ID                                           Criteria        Label  \
1         333  {'N': 40.0, 'P': 73.0, 'K': 20.0, 'temperature...  kidneybeans   
2         364  {'N': 37.0, 'P': 74.0, 'K': 15.0, 'temperature...  kidneybeans   
3         815  {'N': 39.0, 'P': 78.0, 'K': 15.0, 'temperature...       lentil   
4         357  {'N': 31.0, 'P': 79.0, 'K': 25.0, 'temperature...  kidneybeans   
5         816  {'N': 40.0, 'P': 79.0, 'K': 17.0, 'temperature...       lentil   
...       ...                                                ...          ...   
2196     1834  {'N': 8.0, 'P': 6.0, 'K': 33.0, 'temperature':...      coconut   
2197     1823  {'N': 1.0, 'P': 8.0, 'K': 26.0, 'temperature':...      coconut   
2198     1893  {'N': 3.0, 'P': 9.0, 'K': 35.0, 'temperature':...      coconut   
2199     1872  {'N': 1.0, 'P': 6.0, 'K': 35.0, 'temperature':...      coconut   
2200      976  {'N': 0.0, 'P': 5.0, 'K': 36.0, 'temperature':...  pomeg

In [6]:
# Penggunaan
case_base = load_case_base(case_base_file)

# Kriteria baru untuk rekomendasi tanaman
new_case_crop = {
    "N": 11,
    "P": 22,
    "K": 33,
    "temperature": 44.55,
    "humidity": 66.77,
    "ph": 8.8,
    "rainfall": 99.1011
}

# Rekomendasi awal
label = label_crop(case_base, new_case_crop)  # retain sudah dipanggil di dalam fungsi ini jika diperlukan
print(f"Label yang cocok untuk kasus baru {new_case_crop}: {label}")



Tabel Hasil Similaritas dan Ranking:
      Case ID                                           Criteria      Label  \
1        1148  {'N': 7.0, 'P': 17.0, 'K': 26.0, 'temperature'...      mango   
2        1172  {'N': 9.0, 'P': 21.0, 'K': 32.0, 'temperature'...      mango   
3        1105  {'N': 0.0, 'P': 21.0, 'K': 32.0, 'temperature'...      mango   
4        1171  {'N': 18.0, 'P': 20.0, 'K': 26.0, 'temperature...      mango   
5        1115  {'N': 12.0, 'P': 19.0, 'K': 31.0, 'temperature...      mango   
...       ...                                                ...        ...   
2197     1471  {'N': 115.0, 'P': 12.0, 'K': 52.0, 'temperatur...  muskmelon   
2198     1465  {'N': 111.0, 'P': 5.0, 'K': 47.0, 'temperature...  muskmelon   
2199     1485  {'N': 120.0, 'P': 16.0, 'K': 51.0, 'temperatur...  muskmelon   
2200     1404  {'N': 118.0, 'P': 18.0, 'K': 52.0, 'temperatur...  muskmelon   
2201     2201  {'N': 115.0, 'P': 221.0, 'K': 3.0, 'temperatur...   kndbeans   

      Similar